<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [42]:
# Veamos que se cumple que los armónicos esféricos son autofunciones de L2 y Lz

# Primero para L2 se tiene que L2(Ylm) = hbar *l*(l+1)Ylm

for l in range(5):
    for m in range(-l,l+1):
        display (Math(r"\hat{L^{2}}"+f"(Y_{{{l}{m}}}) =" + sp.latex(sp.trigsimp(sp.nsimplify(sp.simplify(
                                                                        sp.together(
                                                                            sp.trigsimp(
                                                                                sp.powsimp(
                                                                                    sp.cancel(
                                                                                        sp.factor(
                                                                                            sp.expand(L2_op(eferic_armonics_numeric(l,m))/eferic_armonics_numeric(l,m))
                                                                                        )
                                                                                    ), force=True
                                                                                )
                                                                            )
                                                                        )
                                                                    ) , rational=True, tolerance=1e-12) )  )))

# Ahora para Lz se tiene que Lz(Ylm) = m*hbar Ylm

print("\n")

for l in range(5):
    for m in range(-l,l+1):
        display (Math(r"\hat{L_{z}}"+f"(Y_{{{l}{m}}}) =" + sp.latex(sp.trigsimp(sp.nsimplify(sp.simplify(
                                                                        sp.together(
                                                                            sp.trigsimp(
                                                                                sp.powsimp(
                                                                                    sp.cancel(
                                                                                        sp.factor(
                                                                                            sp.expand(Lz_op(eferic_armonics_numeric(l,m))/eferic_armonics_numeric(l,m))
                                                                                        )
                                                                                    ), force=True
                                                                                )
                                                                            )
                                                                        )
                                                                    ) , rational=True, tolerance=1e-12) )  )))





<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>